# Example Notebook For Training and Running Zingg Entity Resolution Workflow on Databricks
This notebook runs the Zingg Febrl Example on Databricks. 
Please refer to the 
1. [Zingg Python API](https://readthedocs.org/projects/zingg/) 
2. [Zingg Official Documentation](www.docs.zingg.ai) 
for details.

Please ensure your cluster has the following installed
1. Zingg from pypi
2. Zingg jar from the [repo](https://github.com/zinggAI/zingg/releases)
3. tabular from pypi

Please execute each cell one by one as per the instructions provided.

If you face any issue, please [log an issue](https://github.com/zinggAI/zingg/issues)

You can also join [Zingg's Slack community](https://join.slack.com/t/zinggai/shared_invite/zt-w7zlcnol-vEuqU9m~Q56kLLUVxRgpOA)

# Define locations for the model
The Zingg models and training data are persisted in dbfs. 

**Please edit the model id in the cell below to reflect your model.**

In [0]:
##you can change these to the locations of your choice
##these are the only two settings that need to change
zinggDir = "/models"
modelId = "databricksdemo"

# Setup common functions for use in Zingg. 
These functions setup the internal folders used by Zingg, and help with labeling and training Zingg. 

**No change is needed in the cell below.**

In [0]:

##please leave the following unchanged
MARKED_DIR = zinggDir + "/" + modelId + "/trainingData/marked/"
UNMARKED_DIR = zinggDir + "/" + modelId + "/trainingData/unmarked/"

MARKED_DIR_DBFS = "/dbfs" + MARKED_DIR
UNMARKED_DIR_DBFS = "/dbfs" + UNMARKED_DIR  


import pandas as pd
import numpy as np
 
import time
import uuid
 
from tabulate import tabulate

def cleanModel():
    dbutils.fs.rm(MARKED_DIR, recurse=True)
    # drop unmarked data
    dbutils.fs.rm(UNMARKED_DIR, recurse=True)
    return

# retrieve candidate pairs
def get_candidate_pairs():
  '''
  The purpose of this function is to retrieve candidate pairs that need labeling.
  The function compares the content of the unmarked folder within which the Zingg
  findTraining job deposits candidate paris with those of the marked folder where
  we persist labeled pairs so that no previously labeled pairs are returned.
  '''
  unmarked_pd = pd.DataFrame({'z_cluster':[]})
  marked_pd = pd.DataFrame({'z_cluster':[]})

  
  # read unmarked pairs
  try:
    tmp_pd = pd.read_parquet(
        UNMARKED_DIR_DBFS, 
        engine='pyarrow'
         )
    if tmp_pd.shape[0] != 0: unmarked_pd = tmp_pd
  except:
    pass
  
  # read marked pairs
  try:
    tmp_pd = pd.read_parquet(
        MARKED_DIR_DBFS, 
        engine='pyarrow'
         )
    if tmp_pd.shape[0] != 0: marked_pd = tmp_pd
  except:
    pass
  
  # get unmarked not in marked
  candidate_pairs_pd = unmarked_pd[~unmarked_pd['z_cluster'].isin(marked_pd['z_cluster'])]
  
  return candidate_pairs_pd

# assign label to candidate pair
def assign_label(candidate_pairs_pd, z_cluster, label):
  '''
  The purpose of this function is to assign a label to a candidate pair
  identified by its z_cluster value.  Valid labels include:
     0 - not matched
     1 - matched
     2 - uncertain
  '''
  
  # assign label
  candidate_pairs_pd.loc[ candidate_pairs_pd['z_cluster']==z_cluster, 'z_isMatch'] = label
  
  return
 
# persist labels to marked folder
def save_labels(candidate_pairs_pd):
  '''
  The purpose of this function is to save labeled pairs to the marked folder.
  '''
 
  # make dir if not exists
  dbutils.fs.mkdirs(MARKED_DIR)
  dbutils.fs.ls(MARKED_DIR)
 
  # save labeled data to file
  candidate_pairs_pd.to_parquet(
    MARKED_DIR_DBFS +'/markedRecords_'+ str(time.time_ns()/1000) + '.parquet', 
    compression='snappy',
    index=False # do not include index
    )
  
  return
 
 
def count_labeled_pairs():
  '''
  The purpose of this function is to count the labeled pairs in the marked folder.
  '''
  
  # create initial dataframes
  marked_pd = pd.DataFrame({'z_cluster':[]})
  
  # read unmarked pairs
  try:
    marked_pd = pd.read_parquet(
        MARKED_DIR_DBFS, 
        engine='pyarrow'
         )
  except:
    pass
  
  n_total = len(np.unique(marked_pd['z_cluster']))
  n_positive = len(np.unique(marked_pd[marked_pd['z_isMatch']==1]['z_cluster']))
  n_negative = len(np.unique(marked_pd[marked_pd['z_isMatch']==0]['z_cluster']))
  
  return n_positive, n_negative, n_total

# setup widget 
available_labels = {
    'No Match':0,
    'Match':1,
    'Uncertain':2
    }
dbutils.widgets.dropdown('label', 'Uncertain', available_labels.keys(), 'Is this pair a match?')




# Start building the Zingg program 
The following cell sets up the initial arguments for Zingg. 

**No change is needed in the cell below.**

In [0]:
##this code sets up the Zingg Python interface
from zingg.client import *
from zingg.pipes import *

#build the arguments for zingg
args = Arguments()
# Set the modelid and the zingg dir. You can use this as is
args.setModelId(modelId)
args.setZinggDir(zinggDir)


# Define the input
Our data is in csv so we provide a schema. You can choose other formats like parquet by using Pipe with parquet as the format.
You can also pass in a dataframe by using a Pipe with the in memory format. 
Please refer to [Pipes] (https://zingg.readthedocs.io/en/latest/zingg.html#zingg.pipes.Pipe) for details on different formats

**Please modify this for your data.**

In [0]:
schema = "id string, fname string, lname string, stNo string, add1 string, add2 string, city string, state string, areacode string, dob string, ssn  string"
inputPipe = CsvPipe("testFebrl", "/FileStore/test.csv", schema)

args.setData(inputPipe)

# Configure the output
Here we configure the putput to be a csv, but similar to the input above, the output can be a file format like parquet or delta or a data store like MySQL

**Please modify this for your data.**

In [0]:
#setting outputpipe in 'args'
outputPipe = CsvPipe("resultFebrl", "/tmp/febrlOutput")
args.setOutput(outputPipe)

# Define the match fields and their types

The cell below is used to configure Zingg with the fields for use in matching and the match types.
Details on the field definitions can be found at [Zingg official docs](https://www.docs.zingg.ai)

**Please modify this for your data.**

In [0]:
#set field definitions 
#please change these 
fname = FieldDefinition("fname", "string", MatchType.FUZZY)
lname = FieldDefinition("lname", "string", MatchType.FUZZY)
stNo = FieldDefinition("stNo", "string", MatchType.FUZZY)
add1 = FieldDefinition("add1","string", MatchType.FUZZY)
add2 = FieldDefinition("add2", "string", MatchType.FUZZY)
city = FieldDefinition("city", "string", MatchType.FUZZY)
areacode = FieldDefinition("areacode", "string", MatchType.FUZZY)
state = FieldDefinition("state", "string", MatchType.FUZZY)
dob = FieldDefinition("dob", "string", MatchType.FUZZY)
ssn = FieldDefinition("ssn", "string", MatchType.FUZZY)

fieldDefs = [fname, lname, stNo, add1, add2, city, areacode, state, dob, ssn]
args.setFieldDefinition(fieldDefs)



# Performance settings

The numPartitions define how data is split across the cluster. Please change this as per your data and cluster size by referring to the performance section of the Zingg docs.
The labelDataSampleSize is used for sampling in findTrainingData. It lets Zingg select pairs for labeling in a reasonable amount of time. 
If the findTrainingData phase is taking to much time, please reduce this by atleast 1/10th of its previous value and try again.

**Please modify this for your data.**

In [0]:

# The numPartitions define how data is split across the cluster. 
# Please change the fllowing as per your data and cluster size by referring to the docs.

args.setNumPartitions(4)
args.setLabelDataSampleSize(0.5)



# Select pairs for user labeling
**No change is needed in the cell below.**

In [0]:
options = ClientOptions([ClientOptions.PHASE,"findTrainingData"])

#Zingg execution for the given phase
zingg = ZinggWithSpark(args, options)
zingg.initAndExecute()

# See if we have records for labeling

**No change is needed to the cell below.**

In [0]:
# get candidate pairs
candidate_pairs_pd = get_candidate_pairs()
 
# if no candidate pairs, run job and wait
if candidate_pairs_pd.shape[0] == 0:
  print('No unlabeled candidate pairs found.  Run findTraining job ...')

else:
    # get list of pairs (as identified by z_cluster) to label 
    z_clusters = list(np.unique(candidate_pairs_pd['z_cluster'])) 

    # identify last reviewed cluster
    last_z_cluster = '' # none yet

    # print candidate pair stats
    print('{0} candidate pairs found for labeling'.format(len(z_clusters)))
  

# Label the pairs

A drop down widget is shown which will let the user mark matching, non matching and unsure pairs.

**No change is needed in the cell below.**

In [0]:
  

# get current label setting (which is from last cluster)
last_label = available_labels[dbutils.widgets.get('label')]
 
# assign label to last cluster
if last_z_cluster != '':
  assign_label(candidate_pairs_pd, last_z_cluster, last_label)
 
# get next cluster to label
try:
  z_cluster = candidate_pairs_pd[(candidate_pairs_pd['z_isMatch']==-1) & (candidate_pairs_pd['z_cluster'] != last_z_cluster)].head(1)['z_cluster'].values[0]
except:
  pass
  z_cluster = ''
 
# present the next pair
if z_cluster != '':
  print('IS THIS PAIR A MATCH?')
  print(f"Current widget setting will label this as '{dbutils.widgets.get('label')}'.")
  print('Change widget value if different label required.\n')
  print(
    tabulate(
      candidate_pairs_pd[candidate_pairs_pd['z_cluster']==z_cluster], 
      headers = 'keys', 
      tablefmt = 'psql'
      )
    )
else:
  print('All candidate pairs have been labeled.\n')
 
# hold last items for assignnment in next run
last_z_cluster = z_cluster
 
# if no more to label
if last_z_cluster == '':
  
  # save labels
  save_labels(candidate_pairs_pd)
  
  # count labels accumulated
  n_pos, n_neg, n_tot = count_labeled_pairs()
  print(f'You have accumulated {n_pos} pairs labeled as positive matches.')
  print("If you need more pairs to label, re-run the cell for 'findTrainingData' after saving the labels by running the cell below")

# Save all the labels provided by the user 
No change is needed to the cell below.

In [0]:
# save labels
save_labels(candidate_pairs_pd)

# count labels accumulated
n_pos, n_neg, n_tot = count_labeled_pairs()
print(f'You have accumulated {n_pos} pairs labeled as positive matches.')
print("If you need more pairs to label, re-run the cell for 'findTrainingData'")

# Build the Zingg models and predict matches
No change is needed to the cell below.

In [0]:
options = ClientOptions([ClientOptions.PHASE,"trainMatch"])

#Zingg execution for the given phase
zingg = ZinggWithSpark(args, options)
zingg.initAndExecute()